<a href="https://colab.research.google.com/github/paultgriffiths/boxmox-in-the-cloud/blob/main/run_boxmox_inside_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install bison flex yacc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls
!pwd

In [ ]:
## required modules
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
## suppress output
#%%capture

## flag to check if need to install KPP
install_BOXMOX  = True

## install modules necessary for running BOXMOX and KPP
!apt-get install flex bison

## now install BOXMOX
if install_BOXMOX:
    !cp /content/drive/MyDrive/Colab/Seb-OH/boxmox-1.8.tar.gz ./
    !tar zxf boxmox-1.8.tar.gz
    # install BOXMOX via command-line
    %cd boxmox-1.8
    !././configure --prefix=/content/
    !make
    !make install

In [ ]:
%pwd

In [ ]:
# modify environment variables, NB can't use BASH export, need to use  os module
# see https://stackoverflow.com/questions/49684495/is-it-possible-to-set-environment-variables-in-googles-colaboratory
os.environ['KPP_HOME'] = "/content/share/boxmox/"
os.environ['PATH'] += ":/content/bin/"

# sanity check
!which kpp

In [ ]:
## set up KPP for use with MOZART_T1 mechanism
%cd /content/bin/
!/content/bin/prepare_BOXMOX_mechanism MOZART_4

In [ ]:
#＃ set up new MOZART_T1 PBL experiment
%cd /content/bin/
!/content/bin/new_BOXMOX_experiment_from_example pbl_diurnal_cycle

In [ ]:
## run the first experiment with defaults
%%capture
%cd /content/bin/pbl_diurnal_cycle
!./MOZART_4.exe

In [ ]:
concs = pd.read_csv('./MOZART_4.conc', header=0, delim_whitespace=True, index_col=0)
ems = pd.read_csv('Emissions.csv', skiprows=3, delim_whitespace=True)
rep_ems = pd.concat([ems.iloc[0:24,:]]*4, ignore_index=True)
jrates = pd.read_csv('PhotolysisRates.csv', skiprows=3, delim_whitespace=True)
rep_jrates = pd.concat([jrates.iloc[0:24,:]]*4, ignore_index=True)

In [ ]:
sns.set_context("talk")
fig=plt.figure()
fig = plt.gcf()
fig.set_size_inches(10, 10)



ax = fig.add_subplot(2,2,1)
sns.lineplot( concs['O3'], ax=ax, label='O3')
sns.lineplot( concs['NO2'], ax=ax, label='NO2')
## could go fancier with additional diagnostics
# sns.lineplot( (results['NO2'] + results['O3']), label = 'Oxidant', ax=ax)#.plot(ax=ax, grid=True)
# sns.lineplot( (results['NO2'] + results['O3'] + results['HNO3']), label = 'Oy', ax=ax)#.plot(ax=ax, grid=True)

ax.set(xlabel='Time / hours', ylabel='Mixing ratio / ppm')
plt.grid(True)
plt.legend(ncols=4,bbox_to_anchor=(0.95, -0.15))
plt.title("Pollutants")

ax = fig.add_subplot(2,2,2)
sns.lineplot( rep_ems['CO'], ax=ax, label='CO')
sns.lineplot( rep_ems['NO'], ax=ax, label='NO')
#sns.lineplot( results['NO2'], ax=ax, label='NO2')
## could go fancier with additional diagnostics
# sns.lineplot( (results['NO2'] + results['O3']), label = 'Oxidant', ax=ax)#.plot(ax=ax, grid=True)
# sns.lineplot( (results['NO2'] + results['O3'] + results['HNO3']), label = 'Oy', ax=ax)#.plot(ax=ax, grid=True)

ax.set(xlabel='Time / hours', ylabel='Emissions / cm-2 s-1')
plt.grid(True)
plt.title("Emissions")
plt.legend(ncols=4,bbox_to_anchor=(0.95, -0.15))


ax = fig.add_subplot(2,2,3)
sns.lineplot( rep_jrates['ch3cho'], ax=ax, label='CO')
#sns.lineplot( rep_ems['NO'], ax=ax, label='NO')
#sns.lineplot( results['NO2'], ax=ax, label='NO2')
## could go fancier with additional diagnostics
# sns.lineplot( (results['NO2'] + results['O3']), label = 'Oxidant', ax=ax)#.plot(ax=ax, grid=True)
# sns.lineplot( (results['NO2'] + results['O3'] + results['HNO3']), label = 'Oy', ax=ax)#.plot(ax=ax, grid=True)

ax.set(xlabel='Time / hours', ylabel='Photolysis rate constant / s-1')
plt.grid(True)
plt.title("Sunlight intensity")
plt.legend(ncols=4,bbox_to_anchor=(0.95, -0.15))

plt.tight_layout()

In [ ]:
## optional - plot as function date, rather than hours
# import datetime as dt
# import matplotlib.dates as mdates
# results['time_in_minutes']=results['time']*60
# results['new_time'] = dt.datetime(2010,1,1) + pd.TimedeltaIndex(results['time_in_minutes'], unit='m')
# ... do plotting
# myFmt = mdates.DateFormatter('%d-%H')
# ax.xaxis.set_major_formatter(myFmt)
